<a href="https://colab.research.google.com/github/ParalelaUnsaac/G3-2020-1/blob/main/ProyectoParalela.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!apt-get --purge remove cuda nvidia* libnvidia-*
!dpkg -l | grep cuda- | awk '{print $2}' | xargs -n1 dpkg --purge
!apt-get remove cuda-*
!apt autoremove
!apt-get update

!wget https://developer.nvidia.com/compute/cuda/9.2/Prod/local_installers/cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64 -O cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64.deb
!dpkg -i cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64.deb
!apt-key add /var/cuda-repo-9-2-local/7fa2af80.pub
!apt-get update
!apt-get install cuda-9.2
!pip install git+git://github.com/andreinechaev/nvcc4jupyter.git
%load_ext nvcc_plugin

Reading package lists... Done
Building dependency tree       
Reading state information... Done
Note, selecting 'nvidia-kernel-common-418-server' for glob 'nvidia*'
Note, selecting 'nvidia-325-updates' for glob 'nvidia*'
Note, selecting 'nvidia-346-updates' for glob 'nvidia*'
Note, selecting 'nvidia-driver-binary' for glob 'nvidia*'
Note, selecting 'nvidia-331-dev' for glob 'nvidia*'
Note, selecting 'nvidia-304-updates-dev' for glob 'nvidia*'
Note, selecting 'nvidia-compute-utils-418-server' for glob 'nvidia*'
Note, selecting 'nvidia-384-dev' for glob 'nvidia*'
Note, selecting 'nvidia-libopencl1-346-updates' for glob 'nvidia*'
Note, selecting 'nvidia-driver-440-server' for glob 'nvidia*'
Note, selecting 'nvidia-340-updates-uvm' for glob 'nvidia*'
Note, selecting 'nvidia-dkms-450-server' for glob 'nvidia*'
Note, selecting 'nvidia-kernel-common' for glob 'nvidia*'
Note, selecting 'nvidia-kernel-source-440-server' for glob 'nvidia*'
Note, selecting 'nvidia-331-updates-uvm' for glob 'nvidi

In [ ]:
!nvidia-smi

Tue Oct 27 03:57:05 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.32.00    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   41C    P8     9W /  70W |      0MiB / 15079MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
%%cu
//cuda_runtime_api.h define los funciones y tipos de host públicos para la API CUDA tiempo de ejecución
//cuda_runtime.h define todo cuda_runtime_api.h hace, asi como de tipo incorporado definiciones y superposiciones de funciones para las extensiones de lenguaje CUDA y funciones intrínsecas dispositivo.
#include "cuda_runtime.h"
//Importa / Incluye las funciones, constantes y macros desde la librería de Entrada / Salida estándard (standar input/output
//los periféricos estándar son, monitor, teclado, discoduro, impresora … y esta librería te permite leer desde o enviar información hacia estos periféricos
//como leer variables de ese teclado, o enviar una texto a un archivo en disco.
#include <stdio.h>
//Es el archivo de cabecera de la biblioteca estándar de propósito general de el lenguaje de programación C. Contiene los prototipos de funciones de C para gestión de memoria dinámica, control de procesos y otras. Es compatible con C++ donde se conoce como cstdlib.
#include <stdlib.h>
//cuda.h define el host público funciones y tipos para la API de controlador CUDA
#include <cuda.h>

#define MAXVALUE 10000

//------------------------------------
void numberGen(int N, int max, int* store) //Funcion generar numeros
{
    //Declarando variable entero i
    int i;
    //Tiempo para generar numeros aleatorios
    srand(time(0));
    for (i = 0; i < N; i++)//For para recorrer los numeros
        store[i] = rand() % max; //Almacena en el arreglo
}
//------------------------------------
//Funcion __global__ que se puede llama r desde el host, y se ejecuta en el dispositivo
__global__ void countOdds(int* d, int N, int* odds)
{   
    //Asignamos memoria compartida dinamicamente(extern significa que la matriz hace referencia a la memoria compartida declarada en otro lugar)
    extern __shared__ int count[];

    int myID = blockIdx.x * blockDim.x + threadIdx.x;//Añadiendo el tamaño del grid y del bloque
    int localID = threadIdx.x; // Ponemos el id en cada hilo
    count[localID] = 0;
    if (myID < N)
        count[localID] = (d[myID] % 2); //Un modulo paran detectar si es par o impar
    __syncthreads(); //Sincroniza los subprocesos locales escribiendo en la memoria caché

    // fase de reducción
    int step = 1;
    while (((localID | step) < blockDim.x) && ((localID & step) == 0))
    {
        count[localID] += count[localID | step];
        step <<= 1;
        __syncthreads();
    }

   
    // adiciona al contador global
    if (localID == 0)
        atomicAdd(odds, count[0]);
}

//------------------------------------
int sharedSize(int b)
{
    return b * sizeof(int);
}

//------------------------------------

int main(int argc, char** argv)
{
    //Definimos la cantidad de numeros que se genera aleatoriamente
    int N = 5000;
    //Define punteros host(h*) y device (d*)
    int* ha, * hres, * da, * dres;   // punteros host (h*) y device (d*)

    ha = new int[N];//Declaramos un arreglo en la variable ha
    hres = new int[1];//Declaramos un arreglo en la variable hres
 
    //Almacena un espacio en la memoria da
    cudaMalloc((void**)&da, sizeof(int) * N);
    //Almacena un espacio en la memoria para dres
    cudaMalloc((void**)&dres, sizeof(int) * 1);
 
    //Llama a la funcion numbeGen()
    numberGen(N, MAXVALUE, ha);

    cudaMemcpy(da, ha, sizeof(int) * N, cudaMemcpyHostToDevice);//Copia datos a la memoria del dispositivo
    cudaMemset(dres, 0, sizeof(int));//llenamos los primeros bytes de recuento del área de memoria apuntada por dres con el valor de byte valor constante.

    int blockSize, gridSize;
 
    //Devuelve el tamaño de cuadrícula y bloque que logra la máxima ocupación potencial para una función de dispositivo.
    cudaOccupancyMaxPotentialBlockSizeVariableSMem(&gridSize, &blockSize, (void*)countOdds, sharedSize, N);

    gridSize = ceil(1.0 * N / blockSize);
    printf("Grid : %i    Block : %i\n", gridSize, blockSize);
    countOdds << < gridSize, blockSize, blockSize * sizeof(int) >> > (da, N, dres);

    //Copia datos a la memoria del dispostivo
    cudaMemcpy(hres, dres, sizeof(int), cudaMemcpyDeviceToHost);

   // chequeo
    int oc = 0;
    for (int i = 0; i < N; i++)//Hcae un recorrido
        if (ha[i] % 2){
            printf("%i ",ha[i]);
            oc++;
        }
    printf("")
    //Imprime cantidad de numeros imapres en el arreglo
    printf("\nCantidad de numeros impares - Chequeo\n");
    printf("%i %i\n", *hres, oc);
    cudaFree((void*)da);//Libera memoria de da
    cudaFree((void*)dres);//Libera memoria de dres
 
    delete[]ha; //Operacion eliminar ha
    delete[]hres;//Operacion eliminar hres
    cudaDeviceReset();//Libreria contexto de cuda, quita las asignaciones de todo los dispositivos
    getchar();
    return 0;
}

Grid : 5    Block : 1024
549 3821 3991 3783 1425 4467 9761 4743 1159 1877 8397 2037 3123 7267 3827 9133 7429 6207 7763 461 1295 4679 1057 4535 8893 4809 945 3433 3107 4069 3113 9211 937 7073 7255 7771 4637 1201 1831 5289 2233 1623 4525 3391 6527 3191 1541 2657 343 5949 3023 5407 5403 6777 9397 8609 1039 7447 3737 6391 1715 5313 4235 5397 1707 4941 9601 6705 8239 3547 5871 6617 7627 583 9747 2149 4415 4505 7683 2213 5987 9349 6339 4507 8761 9687 7285 4897 7185 7121 107 1345 9191 5139 741 3397 9411 1751 4125 9269 4911 8549 4759 2193 6617 1105 8427 6349 9919 6955 7433 5857 1081 2091 2345 1821 5069 6997 341 1525 3311 7113 7373 9315 1893 7235 4191 2275 7911 5613 5157 683 1119 841 6349 4335 227 7725 9537 5839 3383 181 8311 9177 1467 6617 6999 8779 2231 4769 5889 1129 3405 6525 6325 991 2513 989 8043 9595 577 8861 1409 911 6993 3589 9855 8967 6531 165 6549 5573 8095 1241 2249 8675 6753 3289 7743 1899 9761 9551 5321 6069 8223 6981 3581 1099 2761 427 809 8955 6695 5135 9063 5349 6951 2227 4695 